In [12]:
import pandas as pd

url = r"sqlite:///C:/Users/blinklet/Documents/Chinook_Sqlite.sqlite"


In [20]:
albums = pd.read_sql_table('Album', url)
print(df.shape)
display(df.head(3))

(347, 3)


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2


In [ ]:
statement = 'SELECT "Album"."Title", "Album"."ArtistId" FROM "Album"'
df = pd.read_sql_query(statement, url)
display(df.head(3))

In [ ]:
statement = 'SELECT "Album"."Title" AS "Album", \
    "Artist"."Name" AS "Artist", \
    "Track"."Name" AS "Track", \
    "Track"."Composer", \
    "Track"."Milliseconds" AS "Length" \
    FROM "Album" \
    JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" \
    JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"'

df = pd.read_sql_query(statement, url)
display(df.head(3))

In [21]:
artists = pd.read_sql_table('Artist', url)
tracks = pd.read_sql_table('Track', url)

In [17]:
statement = """
SELECT name from sqlite_master where type= "table";
"""
tables = pd.read_sql_query(statement, url)
print(tables)

             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track


In [36]:
print(f"albums columns:  {albums.columns}\n")
print(f"artists columns:  {artists.columns}\n")
print(f"tracks columns:  {tracks.columns}")

albums columns:  Index(['AlbumId', 'Title', 'ArtistId'], dtype='object')

artists columns:  Index(['ArtistId', 'Name'], dtype='object')

tracks columns:  Index(['TrackId', 'Name', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer',
       'Milliseconds', 'Bytes', 'UnitPrice'],
      dtype='object')


In [53]:
df1 = (
    albums
    .merge(artists, on='ArtistId')
    .merge(tracks, on='AlbumId', suffixes=['_artist','_track'])
)
print(df1.shape)
display(df1.head(3))

(3503, 12)


,AlbumId,Title,ArtistId,Name_artist,TrackId,Name_track,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock We Salute You,1,AC/DC,1,For Those About To Rock (We Salute You),1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,1,For Those About To Rock We Salute You,1,AC/DC,6,Put The Finger On You,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,1,For Those About To Rock We Salute You,1,AC/DC,7,Let's Get It Up,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99


In [55]:
df1 = (
    albums
    .merge(artists, on='ArtistId')
    .merge(tracks, on='AlbumId', suffixes=['_artist','_track'])
    .rename(columns = {'Title':'Album','Name_artist':'Artist', 
                       'Name_track':'Track', 'Milliseconds':'Length',})
    .drop(['AlbumId', 'TrackId', 'MediaTypeId', 
           'GenreId', 'Bytes', 'UnitPrice', 'ArtistId'], axis=1)
)

print(df1.shape)
display(df1.head(3))

(3503, 5)


,Album,Artist,Track,Composer,Length
0,For Those About To Rock We Salute You,AC/DC,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719
1,For Those About To Rock We Salute You,AC/DC,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662
2,For Those About To Rock We Salute You,AC/DC,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926


In [73]:
customers = pd.read_sql_table('Customer', url)
employees = pd.read_sql_table('Employee', url)

print(f"customers columns:  {customers.columns}\n")
print(f"employees columns:  {employees.columns}")

customers columns:  Index(['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City',
       'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email',
       'SupportRepId'],
      dtype='object')

employees columns:  Index(['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo',
       'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country',
       'PostalCode', 'Phone', 'Fax', 'Email'],
      dtype='object')


In [86]:
df2 = (
    employees
    .merge(customers, left_on='EmployeeId', right_on='SupportRepId', 
           how = 'left', suffixes=['_emp','_cust'])
    .groupby(['EmployeeId','LastName_emp', 'FirstName_emp', 'Title'], 
             as_index=False, dropna=False)['CustomerId']
    .count()
    .rename(columns = {'CustomerId':'Num_Customers'})
)

df2['Employee_Name'] = (
    df2['FirstName_emp'] + ' ' + df2['LastName_emp']
)

df2 = df2.drop(['FirstName_emp', 'LastName_emp'], axis=1)

df2 = df2[['EmployeeId', 'Employee_Name', 'Title', 'Num_Customers']]

print(df2.to_string(index=False))

 EmployeeId    Employee_Name               Title  Num_Customers
          1     Andrew Adams     General Manager              0
          2    Nancy Edwards       Sales Manager              0
          3     Jane Peacock Sales Support Agent             21
          4    Margaret Park Sales Support Agent             20
          5    Steve Johnson Sales Support Agent             18
          6 Michael Mitchell          IT Manager              0
          7      Robert King            IT Staff              0
          8   Laura Callahan            IT Staff              0


In [78]:
display(customers.sample(2))
display(employees.sample(2))


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
53,54,Steve,Murray,None,110 Raeburn Pl,Edinburgh,None,United Kingdom,EH4 1HH,+44 0131 315 3300,None,steve.murray@yahoo.uk,5
16,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5


,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
0,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com


In [84]:
dataframe = pd.merge(
    employees, 
    customers, 
    left_on='EmployeeId', 
    right_on='SupportRepId',
    how='outer',
    suffixes=['_emp','_cust']
)
display(dataframe.sample(3))

,EmployeeId,LastName_emp,FirstName_emp,Title,ReportsTo,BirthDate,HireDate,Address_emp,City_emp,State_emp,...,Company,Address_cust,City_cust,State_cust,Country_cust,PostalCode_cust,Phone_cust,Fax_cust,Email_cust,SupportRepId
24,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,...,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4.0
60,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,...,None,"Calle Lira, 198",Santiago,None,Chile,None,+56 (0)2 635 4444,None,luisrojas@yahoo.cl,5.0
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,...,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3.0


In [90]:
df2 = (
    employees
    .merge(customers, left_on='EmployeeId', 
           right_on='SupportRepId', how = 'outer', 
           suffixes=['_emp','_cust'])
)
display(df2.sample(4))
display(employees)

,EmployeeId,LastName_emp,FirstName_emp,Title,ReportsTo,BirthDate,HireDate,Address_emp,City_emp,State_emp,...,Company,Address_cust,City_cust,State_cust,Country_cust,PostalCode_cust,Phone_cust,Fax_cust,Email_cust,SupportRepId
38,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,...,None,"4, Rue Milton",Paris,None,France,75009,+33 01 49 70 65 65,None,camille.bernard@yahoo.fr,4.0
18,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,...,None,3 Chatham Street,Dublin,Dublin,Ireland,None,+353 01 6792424,None,hughoreilly@apple.ie,3.0
3,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,...,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3.0
36,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,...,None,Rua da Assunção 53,Lisbon,None,Portugal,None,+351 (213) 466-111,None,jfernandes@yahoo.pt,4.0


,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01,2003-10-17,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29,2004-01-02,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09,2004-03-04,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [121]:
statement = """
SELECT 
"Track"."Name" AS "Track_Name", 
"Album"."Title" AS "Album_Title", 
"Artist"."Name" AS "Artist_Name", 
"Track"."Composer" AS "Track_Composer", 
"Track"."Milliseconds" AS "Track_Length", 
"Track"."UnitPrice" AS "Track_Price" 
FROM "Album" 
JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" 
JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"
WHERE
"Artist"."Name" = "Alanis Morissette"
"""

df1 = pd.read_sql_query(statement, url)
display(df1.style.format(thousands=","))

,Track_Name,Album_Title,Artist_Name,Track_Composer,Track_Length,Track_Price
0,All I Really Want,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"284,891",0.990000
1,You Oughta Know,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"249,234",0.990000
2,Perfect,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"188,133",0.990000
3,Hand In My Pocket,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"221,570",0.990000
4,Right Through You,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"176,117",0.990000
5,Forgiven,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"300,355",0.990000
6,You Learn,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"239,699",0.990000
7,Head Over Feet,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"267,493",0.990000
8,Mary Jane,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"280,607",0.990000
9,Ironic,Jagged Little Pill,Alanis Morissette,Alanis Morissette & Glenn Ballard,"229,825",0.990000


In [140]:
statement = 'SELECT "Album"."Title" FROM Album ORDER BY random() LIMIT 5'

albums = pd.read_sql_query(statement, url)
print(albums.shape)
print(albums)

(5, 1)
                                    Title
0                                Presence
1                            Roda De Funk
2  Fauré: Requiem, Ravel: Pavane & Others
3  Os Cães Ladram Mas A Caravana Não Pára
4                       Let There Be Rock


In [133]:
statement = "SELECT * FROM Album"

albums = pd.read_sql_query(statement, url)
print(albums.shape)
print(albums.sample(4))

(347, 3)
     AlbumId                        Title  ArtistId
209      210                Live [Disc 2]       137
195      196  Retrospective I (1974-1980)       128
289      290  The Last Night of the Proms       224
116      117                  Synkronized        92


In [139]:
print(customers.info())
print(employees.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerId    59 non-null     int64 
 1   FirstName     59 non-null     object
 2   LastName      59 non-null     object
 3   Company       10 non-null     object
 4   Address       59 non-null     object
 5   City          59 non-null     object
 6   State         30 non-null     object
 7   Country       59 non-null     object
 8   PostalCode    55 non-null     object
 9   Phone         58 non-null     object
 10  Fax           12 non-null     object
 11  Email         59 non-null     object
 12  SupportRepId  59 non-null     int64 
dtypes: int64(2), object(11)
memory usage: 6.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EmployeeI